In [1]:
import numpy
from simple_modflow.modflow.mf6.sfr import SFR
from pathlib import Path
import pickle
from simple_modflow.modflow.mf6.voronoiplus import VoronoiGridPlus as Vor
import figs as f
import pandas as pd
import flopy
import geopandas as gpd
from simple_modflow.modflow.mf6.mfsimbase import SimulationBase
import shapely as shp
import numpy as np
from simple_modflow.modflow.mf6.recharge import RechargeFromShp as Rch

In [2]:
vor_path = Path(r'C:\Users\lukem\Python\MODFLOW\LakePointe\new_vor_lakepointe.vor')
with open(vor_path, 'rb') as file:
    vor: Vor = pickle.load(file)
with open(Path(r"C:\Users\lukem\Python\MODFLOW\LakePointe\LakePointe.model"),'rb') as file:
    model: SimulationBase = pickle.load(file)

In [3]:
recharge_shp = Path(r"C:\Users\lukem\Python\MODFLOW\LakePointe\inputs\shp\recharge\v2_with_fill\LP_RechargeExJoined_ply_20250529.shp")
daily = Path(r"C:\Users\lukem\Python\MODFLOW\LakePointe\inputs\excel\daily_rch.xlsx")
rch = Rch(model=model, vor=vor, shp=recharge_shp, uid='UID', xlsx_rch=daily, rch_fields_to_pers=list(range(model.nper)), rch_in_vol=True)

In [11]:
rch.get_rch()[0]

[[(0, 153), 0.00013132422150229477],
 [(0, 2090), 0.00013132422150229477],
 [(0, 2255), 0.00013132422150229477],
 [(0, 15403), 4.119989679841597e-05],
 [(0, 15530), 4.119989679841597e-05],
 [(0, 15534), 4.119989679841597e-05],
 [(0, 24237), 4.119989679841597e-05],
 [(0, 24238), 4.119989679841597e-05],
 [(0, 182), 0.00018143903628815513],
 [(0, 183), 0.00018143903628815513],
 [(0, 184), 0.00018143903628815513],
 [(0, 185), 0.00018143903628815513],
 [(0, 407), 0.00018143903628815513],
 [(0, 578), 0.00018143903628815513],
 [(0, 1284), 0.00018143903628815513],
 [(0, 186), 0.00017224647884310683],
 [(0, 187), 0.00017224647884310683],
 [(0, 188), 0.00017224647884310683],
 [(0, 189), 0.00017224647884310683],
 [(0, 408), 0.00017224647884310683],
 [(0, 1000), 0.00017224647884310683],
 [(0, 1001), 0.00017224647884310683],
 [(0, 1006), 0.00017224647884310683],
 [(0, 1007), 0.00017224647884310683],
 [(0, 292), 0.0006320029290226207],
 [(0, 768), 0.0006320029290226207],
 [(0, 820), 0.00063200292902

In [26]:

def recharges():
    """gets a dict of recharge values for each recharge area (each uid) for each stress period. Pass to
    get_rch() to generate a recharge dict to pass to the flopy recharge class."""
    nper = rch.nper
    uids = rch.gdf.index.to_list()
    scaled_rch_fields = rch.rch_fields.mul(rch.shp_to_vor_poly_scale, axis=0)
    rch_fields_dict = scaled_rch_fields.to_dict()
    rch_fields_cols = rch.rch_fields.columns


    recharges = {}
    for uid in uids:
        recharges[uid] = []
    for per in range(nper):
        if rch.fields_to_pers[per] == -1:
            for uid in uids:
                #  if -1 then apply the last field indicated in the provided rch_fields_to_pers agrument
                field_for_per = rch_fields_cols[rch.rch_fields_to_pers[-1]]
                recharges[uid].append(rch_fields_dict[field_for_per][uid])
        elif rch.fields_to_pers[per] == -2:
            for uid in uids:
                recharges[uid].append(rch.background_rch)
        else:
            for uid in uids:
                field_for_per = rch_fields_cols[rch.fields_to_pers[per]]
                recharges[uid].append(rch_fields_dict[field_for_per][uid])
    return recharges


recharges()['NEJC-LK-001']
rch.rch_fields.mul(rch.shp_to_vor_poly_scale, axis=0)

,230401_CRC_cf,230402_CRC_cf,230403_CRC_cf,230404_CRC_cf,230405_CRC_cf,230406_CRC_cf,230407_CRC_cf,230408_CRC_cf,230409_CRC_cf,230410_CRC_cf,...,240322_CRC_cf,240323_CRC_cf,240324_CRC_cf,240325_CRC_cf,240326_CRC_cf,240327_CRC_cf,240328_CRC_cf,240329_CRC_cf,240330_CRC_cf,240331_CRC_cf
UID,,,,,,,,,,,,,,,,,,,,,
CDVY-FR-001,825.046312,13613.264142,8250.463116,0.0,0.0,15675.879921,6187.847337,11550.648363,30114.190374,36302.037711,...,23854.041219,29817.551524,993.918384,2981.755152,4969.591921,44726.327286,25841.877988,993.918384,0.0,0.0
CDVY-IF-001,137.189248,2263.622586,1371.892476,0.0,0.0,2606.595705,1028.919357,1920.649467,5007.407538,6036.326895,...,1972.296879,2465.371099,82.179037,246.537110,410.895183,3698.056649,2136.654953,82.179037,0.0,0.0
CDVY-IF-002,2738.274788,45181.534002,27382.747880,0.0,0.0,52027.220972,20537.060910,38335.847032,99947.029762,120484.090672,...,38182.479013,47728.098766,1590.936626,4772.809877,7954.683128,71592.148149,41364.352264,1590.936626,0.0,0.0
CDVY-LW-001,66.588377,1098.708216,665.883767,0.0,0.0,1265.179158,499.412825,932.237274,2430.475750,2929.888576,...,1411.064701,1763.830877,58.794363,176.383088,293.971813,2645.746315,1528.653426,58.794363,0.0,0.0
CDVY-LW-002,924.264015,15250.356254,9242.640154,0.0,0.0,17561.016292,6931.980115,12939.696215,33735.636562,40667.616677,...,19585.945639,24482.432049,816.081068,2448.243205,4080.405342,36723.648074,21218.107776,816.081068,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWJC-LW-008,495.385036,8173.853098,4953.850362,0.0,0.0,9412.315689,3715.387772,6935.390507,18081.553823,21796.941595,...,10497.632959,13122.041198,437.401373,1312.204120,2187.006866,19683.061797,11372.435705,437.401373,0.0,0.0
SWJC-LW-009,6.214354,102.536840,62.143539,0.0,0.0,118.072725,46.607654,87.000955,226.823918,273.431573,...,131.687479,164.609349,5.486978,16.460935,27.434891,246.914023,142.661436,5.486978,0.0,0.0
SWJC-LW-010,6.751866,111.405788,67.518659,0.0,0.0,128.285452,50.638994,94.526123,246.443106,297.082100,...,195.212421,244.015526,8.133851,24.401553,40.669254,366.023289,211.480123,8.133851,0.0,0.0


In [14]:
vor_polys = vor.gdf_vorPolys
df_intersect = rch.gdf.geometry.apply(lambda geom: vor_polys[vor_polys.intersects(geom)].index.tolist())
df_intersect.name = 'intersect'
df_intersect

UID
NEJC-LK-001    [72, 76, 154, 155, 156, 157, 158, 159, 160, 16...
SWJC-LK-004    [115, 116, 117, 118, 119, 120, 121, 122, 123, ...
SWJC-LK-001                                    [153, 2090, 2255]
NWJC-LK-001    [87, 139, 140, 141, 142, 143, 144, 145, 147, 1...
NWJC-LK-002                  [15403, 15530, 15534, 24237, 24238]
                                     ...                        
CVWD-LW-001    [288, 289, 290, 291, 1347, 1543, 1649, 1677, 1...
LKPT-LW-001    [6653, 6662, 6678, 6681, 6683, 6684, 6689, 669...
LKPT-LW-002    [27, 28, 29, 326, 3590, 4098, 5551, 5916, 5921...
LKPT-LW-003    [22, 8331, 8365, 8369, 8377, 8384, 8388, 8389,...
LKPT-LF-001    [28, 29, 326, 3590, 5405, 5407, 5551, 5570, 55...
Name: intersect, Length: 116, dtype: object

In [38]:
rch.vor_bound_polys['area'] = rch.vor_bound_polys.area
vor_areas = rch.vor_bound_polys.set_index('UID')['area']
daily.apply(lambda x: x / vor_areas, axis=0)

,230401_CRC_cf,230402_CRC_cf,230403_CRC_cf,230404_CRC_cf,230405_CRC_cf,230406_CRC_cf,230407_CRC_cf,230408_CRC_cf,230409_CRC_cf,230410_CRC_cf,...,240322_CRC_cf,240323_CRC_cf,240324_CRC_cf,240325_CRC_cf,240326_CRC_cf,240327_CRC_cf,240328_CRC_cf,240329_CRC_cf,240330_CRC_cf,240331_CRC_cf
UID,,,,,,,,,,,,,,,,,,,,,
CDVY-FR-001,0.000462,0.007620,0.004618,0.0,0.0,0.008775,0.003464,0.006466,0.016857,0.020320,...,0.013353,0.016691,0.000556,0.001669,0.002782,0.025036,0.014465,0.000556,0.0,0.0
CDVY-IF-001,0.003611,0.059574,0.036105,0.0,0.0,0.068600,0.027079,0.050548,0.131785,0.158864,...,0.051907,0.064884,0.002163,0.006488,0.010814,0.097325,0.056233,0.002163,0.0,0.0
CDVY-IF-002,0.010600,0.174899,0.105999,0.0,0.0,0.201399,0.079499,0.148399,0.386897,0.466397,...,0.147805,0.184757,0.006159,0.018476,0.030793,0.277135,0.160122,0.006159,0.0,0.0
CDVY-LW-001,0.000211,0.003479,0.002109,0.0,0.0,0.004007,0.001582,0.002952,0.007697,0.009278,...,0.004468,0.005586,0.000186,0.000559,0.000931,0.008378,0.004841,0.000186,0.0,0.0
CDVY-LW-002,0.000207,0.003410,0.002067,0.0,0.0,0.003927,0.001550,0.002894,0.007544,0.009094,...,0.004380,0.005475,0.000182,0.000547,0.000912,0.008212,0.004745,0.000182,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWJC-LW-008,0.000617,0.010177,0.006168,0.0,0.0,0.011719,0.004626,0.008635,0.022512,0.027138,...,0.013070,0.016337,0.000545,0.001634,0.002723,0.024506,0.014159,0.000545,0.0,0.0
SWJC-LW-009,0.000450,0.007419,0.004497,0.0,0.0,0.008543,0.003372,0.006295,0.016412,0.019785,...,0.009528,0.011911,0.000397,0.001191,0.001985,0.017866,0.010323,0.000397,0.0,0.0
SWJC-LW-010,0.000194,0.003203,0.001941,0.0,0.0,0.003689,0.001456,0.002718,0.007086,0.008542,...,0.005613,0.007016,0.000234,0.000702,0.001169,0.010524,0.006081,0.000234,0.0,0.0


In [10]:
stage: flopy.utils.binaryfile.HeadFile = model.gwf.sfr.output.stage()
budget: flopy.utils.binaryfile.CellBudgetFile = model.gwf.sfr.output.budget()
pd.DataFrame(budget.get_data(kstpkper=(9,8))[0])['q'] / (60*60*24)
stage.get_alldata()

0      -5.003695
1       5.003695
2      -5.006350
3       5.006350
4      -5.008239
         ...    
943    10.572988
944   -10.572971
945    10.572971
946   -10.572224
947    10.572224
Name: q, Length: 948, dtype: float64

In [27]:
for package in model.gwf.get_package_list():
    package = model.gwf.get_package(package)
    print(package.filename)

mf6_model.disv
mf6_model_lak
mf6_model.oc
mf6_model.ic
mf6_model.sto
mf6_model.npf
mf6_model.rch
mf6_model.drn
mf6_model.ghb
mf6_model_lak
mf6_model.sfr


In [4]:
from simple_modflow.modflow.mp3du.particles import ParticleTrackingInput as Pti
pti = Pti(model=model, porosities_by_layer=[0.1, 0.2, 0.1])

{'VELOCITY METHOD LAYER': 3,
 'POROSITY': [0.1, 0.2, 0.1],
 'RETARDATION': 1,
 'DispH': 0,
 'DISPT': 0,
 'DISPV': 0}

In [8]:
stg: flopy.utils.binaryfile.HeadFile = model.gwf.lak.output.stage()
all_stages = pd.Series(stg.get_alldata().squeeze())
nper = len(all_stages)
lake_pump_rates = pd.read_excel(Path(r"C:\Users\lukem\Python\MODFLOW\LakePointe\inputs\excel\lake_pump_rates.xlsx"))
dates = lake_pump_rates.iloc[:nper, 0]
pd.concat([dates, all_stages], axis=1)

,Date,0
0,2023-04-03,337.093587
1,2023-04-04,338.330373
2,2023-04-05,339.486287
3,2023-04-06,340.272236
4,2023-04-07,340.777299
5,2023-04-08,341.274213
6,2023-04-09,341.763737
7,2023-04-10,341.586145
8,2023-04-11,341.006866
9,2023-04-12,341.520660


In [ ]:
vor.gdf_vorPolys.loc[]

In [55]:
cells, cells_df = sfr.get_sorted_cells_along_stream()
stream_cells = vor.gdf_topbtm.loc[cells, :][0].reset_index()
fig = f.Fig()
fig.add_scattergl(x=cells_df.distance_along_stream.to_list(), y=stream_cells[0])
fig.show()

In [37]:
vor.gdf_topbtm.iloc[:, 1:]

,0,1,2,3
0,449.912323,419.487183,376.078400,310.274109
1,449.755707,419.487183,376.078400,310.274109
2,458.352570,416.289429,381.807922,317.640472
3,458.352570,416.170227,381.890228,317.693207
4,453.924164,402.744110,385.000000,320.000000
...,...,...,...,...
11062,399.656464,358.779785,332.096405,289.569275
11063,396.413177,359.034760,332.243042,289.611023
11064,390.442810,375.000000,333.659851,288.763611
11065,534.397644,531.808960,389.947418,311.464050


In [36]:
vor.reconcile_surfaces()


,0,1,2,3
0,449.912323,419.487183,376.078400,310.274109
1,449.755707,419.487183,376.078400,310.274109
2,458.352570,416.289429,381.807922,317.640472
3,458.352570,416.170227,381.890228,317.693207
4,453.924164,402.744110,385.000000,320.000000
...,...,...,...,...
11062,399.656464,358.779785,332.096405,289.569275
11063,396.413177,359.034760,332.243042,289.611023
11064,390.442810,375.000000,333.659851,288.763611
11065,534.397644,531.808960,389.947418,311.464050


In [35]:
# vor.gdf_topbtm.loc[sfr.stream_cells, 0] = vor.gdf_topbtm.loc[sfr.stream_cells, 0] - 2
vor.gdf_topbtm


,geometry,0,1,2,3
0,POINT (1250103.865 750653.361),449.912323,419.487183,376.078400,310.274109
1,POINT (1250109.820 750652.905),449.755707,419.487183,376.078400,310.274109
2,POINT (1252119.913 750775.585),458.352570,416.289429,381.807922,317.640472
3,POINT (1252123.210 750775.921),458.352570,416.170227,381.890228,317.693207
4,POINT (1253038.887 750872.210),453.924164,402.744110,385.000000,320.000000
...,...,...,...,...,...
11062,POINT (1247062.371 745071.543),399.656464,358.779785,332.096405,289.569275
11063,POINT (1247153.266 745014.129),396.413177,359.034760,332.243042,289.611023
11064,POINT (1246819.912 744349.266),390.442810,375.000000,333.659851,288.763611
11065,POINT (1252435.967 747804.615),534.397644,531.808960,389.947418,311.464050


In [12]:
vor.gdf_topbtm.iloc[:, 1:].diff(axis=1)

,0,1,2,3
0,NaN,-30.425140,-43.408783,-65.804291
1,NaN,-30.268524,-43.408783,-65.804291
2,NaN,-42.063141,-34.481506,-64.167450
3,NaN,-42.182343,-34.279999,-64.197021
4,NaN,-51.180054,-17.744110,-65.000000
...,...,...,...,...
11062,NaN,-40.876678,-26.683380,-42.527130
11063,NaN,-37.378418,-26.791718,-42.632019
11064,NaN,-15.442810,-41.340149,-44.896240
11065,NaN,-2.588684,-141.861542,-78.483368


In [47]:
stream_cells = sfr.get_smoothed_reach_elevs(reverse=True)
d = pd.DataFrame.from_dict(stream_cells, orient='index', columns=[0])
vorb = vor.gdf_topbtm.update(d)
vor.gdf_topbtm[0][138]

354.80716

In [48]:
stream_cell_keys = list(stream_cells.keys())
fig =f.Fig()
fig.add_scattergl(x=list(range(len(stream_cells))), y=list(stream_cells.values()))
fig.show()

In [4]:
def generate_lak_period_data(num_stress_periods, lake_ids, lake_stages=None, rainfall_rates=None, evaporation_rates=None, withdrawals=None):
    """
    Generate period data for the MODFLOW 6 LAK package.

    Parameters:
    num_stress_periods (int): Number of stress periods.
    lake_ids (list of int): List of lake IDs.
    lake_stages (list of float, optional): List of lake stages for each stress period.
    rainfall_rates (list of float, optional): List of rainfall rates for each stress period.
    evaporation_rates (list of float, optional): List of evaporation rates for each stress period.
    withdrawals (list of float, optional): List of withdrawal rates for each stress period.

    Returns:
    dict: Dictionary of period data for the LAK package. Each key is a stress period number, and the value is a list of lists.
    """
    period_data = {}

    for period in range(num_stress_periods):
        period_data[period] = []

        for lake_id in lake_ids:
            lake_number = lake_id  # Assuming lake_id is already 0-based for FloPy

            laksetting = [lake_number]
            if lake_stages is not None:
                laksetting.extend(['stage', lake_stages[period]])
            if rainfall_rates is not None:
                laksetting.extend(['rainfall', rainfall_rates[period]])
            if evaporation_rates is not None:
                laksetting.extend(['evaporation', evaporation_rates[period]])
            if withdrawals is not None:
                laksetting.extend(['withdrawal', withdrawals[period]])

            period_data[period].append(laksetting)
    
    return period_data

# Example usage
num_stress_periods = 5  # Example number of stress periods
lake_ids = [0, 1]  # Example lake IDs
lake_stages = [100, 101, 102, 103, 104]  # Example lake stages for each stress period
rainfall_rates = [0.01, 0.02, 0.015, 0.01, 0.02]  # Example rainfall rates for each stress period
evaporation_rates = [0.005, 0.004, 0.006, 0.005, 0.004]  # Example evaporation rates for each stress period
withdrawals = [0.002, 0.003, 0.001, 0.002, 0.003]  # Example withdrawal rates for each stress period
import pandas as pd
lak_period_data = generate_lak_period_data(num_stress_periods, lake_ids, lake_stages, rainfall_rates, evaporation_rates, withdrawals)
pd.DataFrame.from_dict(lak_period_data)


,0,1,2,3,4
0,"[0, stage, 100, rainfall, 0.01, evaporation, 0...","[0, stage, 101, rainfall, 0.02, evaporation, 0...","[0, stage, 102, rainfall, 0.015, evaporation, ...","[0, stage, 103, rainfall, 0.01, evaporation, 0...","[0, stage, 104, rainfall, 0.02, evaporation, 0..."
1,"[1, stage, 100, rainfall, 0.01, evaporation, 0...","[1, stage, 101, rainfall, 0.02, evaporation, 0...","[1, stage, 102, rainfall, 0.015, evaporation, ...","[1, stage, 103, rainfall, 0.01, evaporation, 0...","[1, stage, 104, rainfall, 0.02, evaporation, 0..."
